In [5]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from os import listdir
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

# Displaying images

In [17]:
def imshow (img):
    plt.figure(figsize=(15, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

def imlshow(img):
    plt.figure(figsize=(15, 10))
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    plt.show()
    
def load_images():
    images = []
    img_list = listdir("./img")
    img_list.sort()
    for img_str in img_list:
        images.append(cv2.imread("img/"+img_str))
    return images

## Binarization

In [6]:
BIN_MEAN = cv2.ADAPTIVE_THRESH_MEAN_C
BIN_GAUSS = cv2.ADAPTIVE_THRESH_GAUSSIAN_C

def adapt_bin (img, bin_type, p1, p2):
    return cv2.adaptiveThreshold(img, 255, bin_type, cv2.THRESH_BINARY, p1, p2)

def otsu_gauss_bin (img, kern_size):
    blur = cv2.GaussianBlur(img, (kern_size, kern_size), 0)
    return cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

## Detecting edges with Canny detector

In [7]:
def make_canny(test_img):
    merge_img = np.zeros(test_img.shape[0:2], np.uint32)
    btest_img = cv2.GaussianBlur(test_img, (3,3),0)
    
    counter = 0
    for low_thresh in range(40, 181, 5):
        for high_thresh in range(low_thresh+5, low_thresh+101, 45):
            canny_img = cv2.Canny(test_img, low_thresh, high_thresh)
            gblur_img = cv2.Canny(btest_img, low_thresh, high_thresh)
            canny_img = otsu_gauss_bin(canny_img, 3)
            gblur_img = otsu_gauss_bin(gblur_img, 3)
            merge_img = merge_img + canny_img + gblur_img
            counter = counter + 2
    
    hsv_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)
    sat_img = hsv_img[:,:,1]
    bsat_img = cv2.GaussianBlur(sat_img, (3,3),0)
    val_img = hsv_img[:,:,2]
    
    for low_thresh in range(40, 201, 5):
        for high_thresh in range(low_thresh+5, low_thresh+101, 45):
            canny_sat = cv2.Canny(sat_img, low_thresh, high_thresh)
            canny_val = cv2.Canny(val_img, low_thresh, high_thresh)
            canny_bsat = cv2.Canny(bsat_img, low_thresh, high_thresh)
            canny_sat = otsu_gauss_bin(canny_sat, 3)
            canny_val = otsu_gauss_bin(canny_val, 3)
            canny_bsat = otsu_gauss_bin(canny_bsat, 3)
            merge_img = merge_img + canny_sat + canny_val + canny_bsat
            counter = counter + 3
        
    merge_img = (merge_img / counter).astype(np.uint8)
    merge_img = cv2.threshold(merge_img, 128, 255, cv2.THRESH_BINARY)[1]
    cross_kern = np.array([[0,1,0],[1, 1, 1],[0, 1, 0]], np.uint8)
    sq_kern = np.ones((3,3), np.uint8)
    merge_img = cv2.dilate(merge_img, cross_kern)
    merge_img = cv2.erode(merge_img, sq_kern)
    merge_img = cv2.dilate(merge_img, sq_kern)
    merge_img = cv2.erode(merge_img, cross_kern)
    return merge_img

## Detecting triangle shapes

In [8]:
def scal(fst, sec):
    return fst[0]*sec[0]+fst[1]*sec[0]

def norm2(fst):
    return fst[0]**2+fst[1]**2

def angle(fst, sec):
    return scal(fst, sec)/((norm2(fst)*norm2(sec))**0.5)

def check_contour(cnt):
    leftmost   = np.asarray(cnt[cnt[:,:,0].argmin()][0], dtype=np.float32)
    rightmost  = np.asarray(cnt[cnt[:,:,0].argmax()][0], dtype=np.float32)
    topmost    = np.asarray(cnt[cnt[:,:,1].argmin()][0], dtype=np.float32)
    bottommost = np.asarray(cnt[cnt[:,:,1].argmax()][0], dtype=np.float32)
    
    my_sides = []
    my_sides.append(norm2(leftmost-topmost)**0.5)
    my_sides.append(norm2(leftmost-bottommost)**0.5)
    my_sides.append(norm2(rightmost-topmost)**0.5)
    my_sides.append(norm2(rightmost-bottommost)**0.5)
    my_sides.sort()
    
    my_diags = []
    my_diags.append(norm2(leftmost-rightmost)**0.5)
    my_diags.append(norm2(topmost-bottommost)**0.5)
    my_diags.sort()
    
    if my_sides[1] < 60.0:  # throwing away little blobs
         return False
    
    divisor = my_sides[0]+my_sides[1]  # hypothetical parts of one side
    my_sides /= divisor  # normalising distances in sense of one side
    my_diags /= divisor  # same here
    
    if (my_sides[2]-my_sides[1]-my_sides[0]) < 0.15 and \
       (my_sides[3]-my_sides[2]) < 0.1 and \
        my_diags[0] > 0.866*my_sides[2] and my_diags[0] < 1.1*my_sides[2] and \
        (my_diags[1]-my_sides[3])**2 < 0.01:  # max diag shouldn't differ from side size
        return True
    return False

## Detecting centroids of chips

In [9]:
def mean_point(fst, sec):
    return (fst+sec)/2.0

def find_centroids (contours):
    coords = []
    for cnt in contours:
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        coords.append([cx, cy])
    return coords

## Merging multiple contours into one

In [10]:
def merge_contours(contours):
    centroids = np.asarray(find_centroids(contours))
    delta = 10.0
    clusters = [ [centroids[0]] ]
    contours_to_merge = [ [contours[0]] ]
    added = False
    for centroid_idx, centroid_vec in enumerate(centroids[1:]):
        for cluster_idx, cluster in enumerate(clusters):
            for cluster_vec in cluster:
                if norm2(centroid_vec-cluster_vec)**0.5 < delta:
                    cluster.append(centroid_vec)
                    contours_to_merge[cluster_idx].append(contours[centroid_idx+1])
                    added = True
                    break
            if added is True:
                break
        
        if added is False:
            clusters.append([centroid_vec])
            contours_to_merge.append([contours[centroid_idx+1]])
        else:
            added = False
    
    merged_contours = []
    for cluster_contour in contours_to_merge:
        merged_contours.append(cv2.convexHull(np.vstack(cluster_contour)))
    return merged_contours

## Cutting image of triangle

In [11]:
def get_contour_img(img, cnt, need_to_erode=True):
    start = (cnt[:,:,0].min(), cnt[:,:,1].min())
    end = (cnt[:,:,0].max(), cnt[:,:,1].max())
    mask_img = np.zeros(img.shape[0:2], np.uint8)
    ROIpoints = cv2.approxPolyDP(cnt, 5.0, True)
    mask_img = cv2.fillConvexPoly(mask_img, cnt, 255, 8, 0)
    if need_to_erode is True:
        mask_img = cv2.erode(mask_img, np.ones((9, 9), np.uint8))
    
    mask_img = mask_img[start[1]:end[1], start[0]:end[0]]
    img_cut = img[start[1]:end[1], start[0]:end[0]].copy()
    img_cut[:,:,0] = np.minimum(img_cut[:,:,0], mask_img)
    img_cut[:,:,1] = np.minimum(img_cut[:,:,1], mask_img)
    img_cut[:,:,2] = np.minimum(img_cut[:,:,2], mask_img)
    
    return img_cut

## Separating corners of triangle

In [12]:
def get_corners(img):
    test_img = img.copy()
    bin_img = make_canny(test_img)
    cnt_img, contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    my_contour = cv2.convexHull(np.vstack(contours)) # only one contour left 
    eps = 0.07*cv2.arcLength(my_contour,True)
    poly_pts = cv2.approxPolyDP(my_contour, eps, True)
    med_pts = []
    for i in range(0, 3):
        med_pts.append( np.array((poly_pts[i]+poly_pts[(i+1) % 3])/2.0, np.int32))
    
    M = cv2.moments(my_contour)
    center = np.asarray([[int(M['m10']/M['m00']), int(M['m01']/M['m00'])]], np.int32)
    
    corner_images = []
    for i in range(0, 3):
        corner_cnt = np.array([poly_pts[i], med_pts[i], center, med_pts[(i+2)%3]], np.int32)
        corner_images.append(get_contour_img(test_img, corner_cnt.astype(np.int32), False))
    return corner_images

## Building dot recognition models

In [13]:
def train_my_models():
    dots_data = np.asarray(pd.read_csv("dots.csv"))  # sample set, that was built by myself
    np.random.shuffle(dots_data)  # shuffling data

    dots_X = dots_data[:, 1:]  # separating X and y
    dots_y = dots_data[:, 0].ravel()
    dots_y2 = dots_y.copy()
    dots_y2[dots_y > 0] = 1
    # normalising all features according to their ranges
    dots_X[:, :3] = dots_X[:, :3]/255.0  # RGB components
    dots_X[:,  3] = dots_X[:,  3]/360.0  # hue component (angle up to 360)
    dots_X[:, 4:] = dots_X[:, 4:]/100.0  # saturation and value components (percentage)

    dots_X2 = dots_X[dots_y > 0, :].copy()
    dots_y3 = dots_y[dots_y > 0].copy()
    
    knn = KNeighborsClassifier(9)
    knn.fit(dots_X, dots_y2)

    knn2 = KNeighborsClassifier(5)
    knn2.fit(dots_X2, dots_y3)
    
    return (knn, knn2)

def make_dataset(img, is_xgboost):
    #dividing img on channels
    test_img = img.copy()
    hsv_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    # creating dataset and copying columns
    dataset = np.zeros((test_img.shape[0]*test_img.shape[1], 6))
    dataset[:,:3] = test_img.reshape((-1, 3))
    dataset[:,3:] = hsv_img.reshape((-1, 3))
    # normalising data
    dataset[:, :3] = dataset[:, :3]/255.0  # RGB components
    dataset[:,  3] = dataset[:,  3]/180.0  # hue component (angle up to 360)
    dataset[:, 4:] = dataset[:, 4:]/255.0  # saturation and value components (percentage)
    
    if is_xgboost is True:
        dataset = xgb.DMatrix( dataset)
    return dataset

def detect_dots(img, model1, model2, is_xgboost1=False, is_xgboost2=False):
    test_img = img.copy()
    dataset = make_dataset(test_img, is_xgboost1)
    img_y = model1.predict(dataset)  # predicting foreground/background
    
    test_img = img.copy()
    img_y = img_y.reshape((test_img.shape[0], test_img.shape[1]))
    test_img[img_y == 0.0] = [0, 0, 0]
    
    dataset = make_dataset(test_img, is_xgboost2)
    img_y2 = model2.predict(dataset)
    
    test_img = img.copy()
    img_y2 = img_y2.reshape((test_img.shape[0], test_img.shape[1]))
    test_img[img_y2 == 1.0] = [255, 255, 255]
    test_img[img_y2 == 2.0] = [0, 255, 0]
    test_img[img_y2 == 3.0] = [0, 255, 255]
    test_img[img_y2 == 4.0] = [255, 255, 0]
    test_img[img_y2 == 5.0] = [0, 0, 255]
    test_img[img_y == 0.0] = [0, 0, 0]
    
    cross_kern = np.asarray([[0, 1, 0], [1, 1, 1], [0, 1, 0]], np.uint8)
    test_img = cv2.morphologyEx(test_img, cv2.MORPH_OPEN, cross_kern)
    test_img = cv2.morphologyEx(test_img, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))
    test_img = cv2.morphologyEx(test_img, cv2.MORPH_OPEN, cross_kern)
    test_img = cv2.morphologyEx(test_img, cv2.MORPH_ERODE, np.ones((3, 3), np.uint8))
    return test_img

## Counting dots in each corner

In [14]:
def count_dots(img):
    clusters = [0 for i in range(5)]
    clus_vals = [np.array([255, 255, 255], np.uint8),
                 np.array([0, 255, 0], np.uint8),
                 np.array([0, 255, 255], np.uint8),
                 np.array([255, 255, 0], np.uint8),
                 np.array([0, 0, 255], np.uint8)]
    back_val = np.array([0, 0, 0], np.uint8)
    for row in img:
        for dot in row:
            if (dot == back_val).all():
                continue
            for i, clus_val in enumerate(clus_vals):
                if (dot == clus_val).all():
                    clusters[i] += 1

    #defining patterns
    m = max(clusters)
    if m == 0:
        return 0
    elif (clusters[3] >= clusters[0] or clusters[1] >= clusters[0])\
         and clusters[0] > 1 and clusters[2] < clusters[0] and\
        clusters[4] < clusters[0]:
        return 1
    
    return ([i for i, j in enumerate(clusters) if j >= m][0]+1)      

## Final function

In [19]:
def detect_chips(img):
    # detecting triangles
    test_img = img.copy()
    bin_img = make_canny(test_img)
    c_img, contours, h = cv2.findContours(bin_img, cv2.RETR_TREE,\
                                          cv2.CHAIN_APPROX_SIMPLE)
    new_contours = []
    for contour in contours:
        if check_contour(contour) is True:
            new_contours.append(contour)

    m_contours = merge_contours(new_contours)
    centroids = find_centroids(m_contours)
    
    #detecting dots
    cnt_images = []
    for cnt in m_contours:
        cnt_img = get_contour_img(test_img, cnt)
        cnt_images.append(cnt_img)
    
    crn_images = []
    for cnt_img in cnt_images:
        crn_images.append(get_corners(cnt_img))
        
    model1, model2 = train_my_models()
    
    all_dots = []
    for idx, crn_list in enumerate(crn_images):
        dots_list = []
        for crn_img in crn_list:
            counted = count_dots(detect_dots(crn_img, model1, model2))
            dots_list.append(counted)
        dots_list.sort()
        all_dots.append(dots_list)
        
    return (len(centroids), list(zip(centroids, all_dots)))

In [21]:
images = load_images()
output = detect_chips(images[1])

In [27]:
f = open("out.txt", "w")
print(output[0], file=f) # N
for entry in output[1]:
    print(entry[0][0],", ",entry[0][1],"; ",entry[1][0],", ",entry[1][1],", ",entry[1][2], sep="", file=f)
f.close()